In [ ]:
# !pip install pytorch-lightning
# !pip install lightning-bolts
!brew install pytorch
!brew install pytorchvision

pytorch 2.0.0 is already installed but outdated (so it will be upgraded).
==> Fetching dependencies for pytorch: libuv, mpfr, numpy, protobuf@21, pybind11, python-typing-extensions, pyyaml and libomp
==> Fetching libuv
==> Downloading https://ghcr.io/v2/homebrew/core/libuv/manifests/1.46.0
######################################################################### 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libuv/blobs/sha256:8c3beb4d11ed
######################################################################### 100.0%
==> Fetching mpfr
==> Downloading https://ghcr.io/v2/homebrew/core/mpfr/manifests/4.2.0-p12
######################################################################### 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/mpfr/blobs/sha256:43db595106704
######################################################################### 100.0%
==> Fetching numpy
==> Downloading https://ghcr.io/v2/homebrew/core/numpy/manifests/1.25.1
########################################

In [1]:
import torch
import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F
from pl_bolts.models.autoencoders.components import (
    resnet18_decoder,
    resnet18_encoder,
)

class VAE(pl.LightningModule):
    def __init__(self, enc_out_dim=512, latent_dim=256, input_height=178):
        super().__init__()

        self.save_hyperparameters()

        # encoder, decoder
        self.encoder = resnet18_encoder(False, False)
        self.decoder = resnet18_decoder(
            latent_dim=latent_dim,
            input_height=input_height,
            first_conv=False,
            maxpool1=False
        )

        # distribution parameters
        self.fc_mu = nn.Linear(enc_out_dim, latent_dim)
        self.fc_var = nn.Linear(enc_out_dim, latent_dim)

        # for the gaussian likelihood
        self.eps = nn.Torch.Distributions.Normal(0, 1)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

    def gaussian_likelihood(self, mean, logscale, sample):
        scale = torch.exp(logscale)
        dist = torch.distributions.Normal(mean, scale)
        log_pxz = dist.log_prob(sample)
        return log_pxz.sum(dim=(1, 2, 3))

    def kl_divergence(self, z, mu, std):
        # --------------------------
        # Monte carlo KL divergence
        # --------------------------
        # 1. define the first two probabilities (in this case Normal for both)
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        # 2. get the probabilities from the equation
        log_qzx = q.log_prob(z)
        log_pz = p.log_prob(z)

        # kl
        kl = (log_qzx - log_pz)
        kl = kl.sum(-1)
        return kl

    def training_step(self, batch, batch_idx):
        x = batch

        #encode 
        x_encoded = self.encoder(x)
        mu, sigma = self.fc_mu(x_encoded), torch.exp(self.fc_var(x_encoded))

        # get (sample) z from dist normal (q)
        q = self.eps.sample(mu, sigma)
        z = mu + sigma* q

        # decode
        x_hat = self.decoder(z)

        #loss
        recon_loss = torch.nn.MSELoss(x,x_hat)
        kl = self.kl_divergence(z, mu, sigma)
        elbo = (kl - recon_loss)
        elbo = elbo.mean()

        self.log_dict({
            'elbo': elbo,
            'kl': kl.mean(),
            'recon_loss': recon_loss.mean(), 
            'kl': kl.mean(),
        })

        return elbo

/Users/robertomartins/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'pytorch_lightning'

In [30]:
import torchvisio

import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn.functional as F


path = './img_align_celeba/'
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_labels = os.listdir(img_dir)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        full_path = str(idx).rjust(6, '0') + '.jpg'
        img_path = os.path.join(self.img_dir, full_path)
        image = read_image(img_path)
        image = image.unsqueeze(0)
        image = F.interpolate(image, size=(3, 32, 32), mode='bilinear', align_corners=False)
        image = image.squeeze(0)
        image = image.float()
        if self.transform:
            image = self.transform(image)
        return image



path = './img_align_celeba/'
dataset = CustomImageDataset(path)
celeba = DataLoader(dataset, batch_size=32, shuffle=True)

ModuleNotFoundError: No module named 'torchvisio'

In [27]:
pl.seed_everything(1234)

vae = VAE()
trainer = pl.Trainer(max_epochs=7, accelerator='mps')
trainer.fit(vae, celeba)

Global seed set to 1234
/var/folders/z7/lcnrptz91gsbvkk8gsb76dlh0000gn/T/ipykernel_22168/3548631254.py:17: UnderReviewWarning: The feature resnet18_encoder is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  self.encoder = resnet18_encoder(False, False)
/Users/robertomartins/.pyenv/versions/3.11.1/lib/python3.11/site-packages/pl_bolts/models/autoencoders/components.py:334: UnderReviewWarning: The feature ResNetEncoder is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  return ResNetEncoder(EncoderBlock, [2, 2, 2, 2], first_conv,

Training: 0it [00:00, ?it/s]

ValueError: Input and output must have the same number of spatial dimensions, but got input with spatial dimensions of [218, 178] and output size of (3, 32, 32). Please provide input tensor in (N, C, d1, d2, ...,dK) format and output size in (o1, o2, ...,oK) format.

In [ ]:
from matplotlib.pyplot import imshow, figure
import numpy as np
from torchvision.utils import make_grid
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
figure(figsize=(8, 3), dpi=300)

# Z COMES FROM NORMAL(0, 1)
num_preds = 16
p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
z = p.rsample((num_preds,))

# SAMPLE IMAGES
with torch.no_grad():
    pred = vae.decoder(z.to(vae.device)).cpu()

# UNDO DATA NORMALIZATION
normalize = cifar10_normalization()
mean, std = np.array(normalize.mean), np.array(normalize.std)
img = make_grid(pred).permute(1, 2, 0).numpy() * std + mean

# PLOT IMAGES
imshow(img)